In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [2]:
query = f"""

with t1 as 
(select 
    distinct customer_id, gender, age, taxi_lifetime_rides as ltr, taxi_lifetime_last_ride_city as taxi_city
from datasets.iallocator_customer_metrics
where run_date = '2022-12-12'
    and taxi_rpc_last_21_days > 0
    and taxi_lifetime_rides > 7
    and taxi_lifetime_last_ride_city in ('Bangalore','Hyderabad','Delhi','Chennai','Mumbai','Kolkata','Jaipur')
group by 1,2,3,4,5
),

t2 as (
select 
    distinct customer_id,taxi_gc_tag,taxi_recency_segment,taxi_lifetime_stage
from datasets.iallocator_customer_segments
where run_date= '2022-12-12'
group by 1,2,3,4
)
select t1.*, t2.taxi_gc_tag,taxi_recency_segment,taxi_lifetime_stage
from t1 join t2
on t1.customer_id=t2.customer_id

"""

In [3]:
df = pd.read_sql(query, conn)
df

,customer_id,gender,age,ltr,taxi_city,taxi_gc_tag,taxi_recency_segment,taxi_lifetime_stage
0,62f46151c30fe23b7a8d3765,male,None,83,Hyderabad,GC,RECENT,DETOX
1,5b7f77c6b5a9cf060c666dc0,male,None,49,Hyderabad,GC,STATIONARY,SOFT_CHURN
2,6084e220c14f0f101cacf8eb,male,None,15,Hyderabad,NON_GC,STATIONARY,CHURN_OTB
3,634cca0776a9b32aa8aa58fb,unknown,None,12,Hyderabad,NON_GC,RECENT,HOOK
4,632dd3d4673c23254b6362c6,female,None,45,Hyderabad,GC,RECENT,COMMITTED
...,...,...,...,...,...,...,...,...
2433465,624fc795556388189992e11f,female,None,18,Delhi,NON_GC,STATIONARY,SOFT_CHURN
2433466,5db58f34f53c582142666ec2,male,None,97,Hyderabad,NON_GC,RECENT,HOOK
2433467,603672b6f5d36b5eacf76958,male,None,24,Chennai,GC,RECENT,COMMITTED
2433468,5e10265da10a71228f8d0b12,male,None,43,Hyderabad,NON_GC,RECENT,COMMITTED


In [4]:
df_Bangalore = df[df['taxi_city'].isin(['Bangalore'])]
df_Hyderabad = df[df['taxi_city'].isin(['Hyderabad'])]
df_Delhi = df[df['taxi_city'].isin(['Delhi'])]
df_Mumbai = df[df['taxi_city'].isin(['Mumbai'])]
df_Chennai = df[df['taxi_city'].isin(['Chennai'])]
df_Kolkata = df[df['taxi_city'].isin(['Kolkata'])]
df_Jaipur = df[df['taxi_city'].isin(['Jaipur'])]

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
X = df_Bangalore # Contains all columns .
y = df_Bangalore[['taxi_lifetime_stage','taxi_gc_tag']] # Dataframe of just the column on which to stratify.

# Split original dataframe into train and test dataframes.
df_train, df_test, y_train, y_test = train_test_split(X,
                                                      y,
                                                      stratify=y,
                                                      test_size=(0.5),
                                                      random_state=None)

# Split train into two groups to create test 1 and test 2
df_t1, df_t2, y_t1, y_t2 = train_test_split(df_train,
                                                      y_train,
                                                      stratify=y_train,
                                                      test_size=(0.5),
                                                      random_state=None)


# Split test into two groups to create test 3 and test 4
df_t3, df_t4, y_t3, y_t4 = train_test_split(df_test,
                                                      y_test,
                                                      stratify=y_test,
                                                      test_size=(0.5),
                                                      random_state=None)

df_t1['group_tc']='test_1'
df_t2['group_tc']='test_2'
df_t3['group_tc']='test_3'
df_t4['group_tc']='test_4'

#Combined table to be uploaded in GCP for tracking export it without headers
df_final=df_t1.append([df_t2,df_t3,df_t4]).reset_index(drop=True)

ModuleNotFoundError: No module named 'sklearn'

In [14]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.
